In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 20 13:31:18 2024

Calculate the size of the training data for each run, because I forgot top save this value when running the model
"""
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import my_utils
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
from sklearn.model_selection import GroupShuffleSplit
import datetime
from tensorflow import keras

def split_data_black_white_ratio_in_loop (df_group1, df_group2, ratio):

    """
    split data depending on the ratio of PtIDs from each group
    
    Parameters
    ----------
    df_group1 : Dataframe
        df containing data from group1
    df_group2 : Dataframe
        df containing data from group2.
    ratio : TYPE
        DESCRIPTION.

    Returns
    -------
    x_train : TYPE
        data to train on.
    y_train : TYPE
        labels for data to train on.
    x_val : TYPE
        data to validate on (or test).
    y_val : TYPE
        labels for data to validate on (or test).

    """
    # split based on PtID
    if percentage == 100: #  if there are no patients in group 2
    
        gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
        train_idx, val_test_idx = next(gss.split(df_group1, groups=df_group1['PtID']))
    
        x_train_temp1 = df_group1.iloc[train_idx]
        x_val_temp1 = df_group1.iloc[val_test_idx]
    
        x_train, y_train, x_val, y_val = my_utils.seperate_the_target(x_train_temp1, x_val_temp1)
    
    elif percentage == 0: # if there are no patients in group 1
     
        gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42) 
        train_idx, test_idx = next(gss.split(df_group2, groups=df_group2['PtID']))
    
        x_train_temp2 = df_group2.iloc[train_idx]
        x_val_temp2 = df_group2.iloc[test_idx]
    
        x_train, y_train, x_val, y_val = my_utils.seperate_the_target(x_train_temp2, x_val_temp2)
        
    else: # if there are patients in group 2
        x_train, y_train, x_val, y_val = my_utils.get_groupShuflesplit_equal_groups(df_group1, df_group2,test_size=0.2, seperate_target=True)     # split based on PtID
    
    return x_train, y_train, x_val, y_val


2024-08-06 15:55:20.397218: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-06 15:55:20.436652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 15:55:20.436732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 15:55:20.437726: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-06 15:55:20.444361: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-06 15:55:20.445227: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# group = 'race'
# group = 'gender'
# group = 'age'

# path = r'/Users/au605715/Documents/GitHub/study1/1_2_model_input_ws60min_ph60min_v6.csv'
# df = pd.read_csv(path)



# # Specify the file path

# file_path = f"/Users/au605715/Documents/GitHub/study1/1_3_data_split_{group}_v6_2.pkl"

# # Read from file
# with open(file_path, 'rb') as file:
#     dictionary = pickle.load(file)
    
    
group =  'race'    
group_name = "Race"

#                    load data
file_path_df = r'../results/preprocessed_data/1_2_model_input_ws60min_ph60min_v6.csv'
df = pd.read_csv(file_path_df)



# Specify the file path
file_path = f"../results/preprocessed_data/1_3_data_split_{group}_v6.pkl"

# Read from file
with open(file_path, 'rb') as file:
    dictionary = pickle.load(file)

In [3]:
#%%                     extract data from dictionary
testing_mode = False


df.drop(columns=['Gender','EduLevel','ageAtEnroll'], inplace = True) # Gender
df.dropna(inplace=True)


In [4]:
dict_results = {}

i = 0

for (PtID, percentage), value in dictionary.items():
    print(i)
    i=i+1
    
#                     get data from dictionary

    ptid_training_gr1 = value['training_gr1']
    ptid_training_gr2 = value['training_gr2']
    ptid_test = value['PtID_test']
    ptid_group = value['group']

    df_train_gr1 = df[df['PtID'].isin(ptid_training_gr1)]
    df_train_gr2 = df[df['PtID'].isin(ptid_training_gr2)]
    df_test = df[df['PtID']==ptid_test]
    


#%%                split dataset80/20 
    
    # x_train, y_train, x_val, y_val = split_data_group_ratio_in_loop(df_train_m, df_train_f, percentage)
   

#%%                     split dataset using one week of data to validate on the rest

    df_train_gr12 = pd.concat([df_train_gr1, df_train_gr2], ignore_index=True)
    df_train_gr12.reset_index

    # split within patients, train/val for General model
    x_train, y_train, x_val, y_val = my_utils.split_within_PtID(df_train_gr12, numb_values_to_remove=-672, seperate_target=True, group_column=group_name) # split witin  PtID: 4values/hour * 24hour/day * 7days/week = 672 values/week

    print('ged 1')
    #%%                   Fine- tuning: split data
    

    # split within patients, train/test
    xy_train_tl, xy_test_tl = my_utils.split_within_PtID(df_test, numb_values_to_remove=-672, seperate_target=False, group_column=group_name) # split witin  PtID: 4values/hour * 24hour/day*7days/week = 672 values/week

    # baseline_test_tl = xy_test_tl['Value_4']
    print('ged 2')    

    # split train in train/val with seperate targets
    x_train_tl, y_train_tl, x_val_tl, y_val_tl = my_utils.split_time_series_data(xy_train_tl, test_size=0.15, group_column=group_name)
    print('ged 3')
    
    # seperate target from test
    x_test_tl, y_test_tl = my_utils.seperate_the_target(xy_test_tl, group_column=group_name)
    print('ged 4')
    
    #%% My actual/true values and my baseline value
    
    y_actual_tl = y_test_tl
    # y_last_val_tl = baseline_test_tl.to_numpy()
    
    #%% train size
    train_size_single_tl = len(x_train_tl)
    train_size_base = len(x_train)
    train_size_naive = len(x_test_tl)
    
#%% 

    dict_results[(PtID, percentage)] = {
        "PtId_test": ptid_test,
        "ptid_group": ptid_group,
        "y_actual": y_test_tl,
        "y_last_val": xy_test_tl['Value_4'],
        "train_size_single_tl": train_size_single_tl,
        "train_size_base": train_size_base,
        "train_size_naive": train_size_naive
        } 
    
    
    if testing_mode==True:
        break
    # qif testing_mode == False:
    #%%
    

        # file_path = f"/home/hbt/jchr_data/jchr_racial_diff/results/processed_data/2_1_1_predicted_results_rnn_v6_race2/patient{PtID}_ratio{percentage}.pkl"
        
        
        # with open(file_path, 'wb') as file:
        #     # Serialize and save the list to the file
        #     pickle.dump(dict_results, file)
        
        # dict_results = {}

0
ged 1
ged 2
ged 3
ged 4
1
ged 1
ged 2
ged 3
ged 4
2
ged 1
ged 2
ged 3
ged 4
3
ged 1
ged 2
ged 3
ged 4
4
ged 1
ged 2
ged 3
ged 4
5
ged 1
ged 2
ged 3
ged 4
6
ged 1
ged 2
ged 3
ged 4
7
ged 1
ged 2
ged 3
ged 4
8
ged 1
ged 2
ged 3
ged 4
9
ged 1
ged 2
ged 3
ged 4
10
ged 1
ged 2
ged 3
ged 4
11
ged 1
ged 2
ged 3
ged 4
12
ged 1
ged 2
ged 3
ged 4
13
ged 1
ged 2
ged 3
ged 4
14
ged 1
ged 2
ged 3
ged 4
15
ged 1
ged 2
ged 3
ged 4
16
ged 1
ged 2
ged 3
ged 4
17
ged 1
ged 2
ged 3
ged 4
18
ged 1
ged 2
ged 3
ged 4
19
ged 1
ged 2
ged 3
ged 4
20
ged 1
ged 2
ged 3
ged 4
21
ged 1
ged 2
ged 3
ged 4
22
ged 1
ged 2
ged 3
ged 4
23
ged 1
ged 2
ged 3
ged 4
24
ged 1
ged 2
ged 3
ged 4
25
ged 1
ged 2
ged 3
ged 4
26
ged 1
ged 2
ged 3
ged 4
27
ged 1
ged 2
ged 3
ged 4
28
ged 1
ged 2
ged 3
ged 4
29
ged 1
ged 2
ged 3
ged 4
30
ged 1
ged 2
ged 3
ged 4
31
ged 1
ged 2
ged 3
ged 4
32
ged 1
ged 2
ged 3
ged 4
33
ged 1
ged 2
ged 3
ged 4
34
ged 1
ged 2
ged 3
ged 4
35
ged 1
ged 2
ged 3
ged 4
36
ged 1
ged 2
ged 3
ged 4
37
ged 1
ge

In [5]:
# After collecting data, create DataFrame
# df = pd.DataFrame(data)
#%%

import pandas as pd
df_results = pd.DataFrame.from_dict(dict_results, orient='index').reset_index()
df_results = df_results.rename(columns={'level_0': 'PtID', 'level_1': 'ratio'})
df_results.drop(columns = ['y_actual', 'y_last_val'], axis=1,inplace=True)


# path_save = rf'../results/processed_data/3_1_calculated_results/train_size2_{group}.csv'
# df_results.to_csv(path_save)

In [6]:
df_results.head()

,PtID,ratio,PtId_test,ptid_group,train_size_single_tl,train_size_base,train_size_naive
0,179,100,179,child,2925,537599,672
1,45,100,45,child,5054,535094,672
2,88,100,88,child,4511,535733,672
3,202,100,202,child,4307,535973,672
4,139,100,139,adult,3124,533913,672
